# JSON 사용하기

In [1]:
import json

In [2]:
a = {'value1': 5, 'value2':10, 'seq':[1,2,3,4,5]}

#test.json이라는 파일로 저장
filename = 'test.json'
#file object를 f라고 지정
with open(filename, 'w') as f:
#파일 f에 a라는 데이터를 덮어씌워라
    json.dump(a, f)

#with 없이 열면 f.close()를 해주어야 하는데 까먹는 경우가 많아서
#with를 쓰고 열면 이 문구를 벗어나면 자동으로 close해줌 => 안전함
with open(filename, 'r') as f:
    result = json.load(f)
    print(result)
    print(result['seq'])

{'value1': 5, 'value2': 10, 'seq': [1, 2, 3, 4, 5]}
[1, 2, 3, 4, 5]


In [3]:
!ls
!cat test.json
# 딕셔너리 형태로 출력이 되어있음
# JSON -> 그냥 딕셔너리를 하드디스크에 편리하게 저장했다가 불러들이는 것이다.

'json&hash.ipynb'   test.json
{"value1": 5, "value2": 10, "seq": [1, 2, 3, 4, 5]}

# Hash 사용하기


In [4]:
import hashlib

a = "my name is jaeyoung"
hash_key = hashlib.sha1(a.encode()).hexdigest()[:6]
print(hash_key)

37b0f4


In [5]:
# setting : 실험결과가 담긴 딕셔너리
setting = {'value1': 5, 'value2':10, 'seq':[1,2,3,4,5]}

#str(setting) setting에 있는 값들을 문자로 바꿔줌
print(str(setting))

#실험 setting 값에 대한 hash code를 생성
hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
print(hash_key)

{'value1': 5, 'value2': 10, 'seq': [1, 2, 3, 4, 5]}
c40aaa


In [6]:
#파일 이름 만들어보기

setting = {'value1': 6, 'value2':10, 'seq':[1,2,3,45], 'exp_name': 'exp1'}
exp_name = setting['exp_name']
hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
filename = '{}-{}.json'.format(exp_name, hash_key)
print(filename)

exp1-0a7972.json


# 실제 예시 코드

In [7]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

#setting : hyperparameter들이 담겨있는 딕셔너리
def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    #epoch가 달라져도 해쉬값이 달라짐
    #test_batch_size는 실험 결과에 영향이 전혀 없는데 해쉬 값이 달라짐
    #del 명령어로 해당 값은 dictionary에서 지워줌
    del setting['epoch']
    del setting['test_batch_size']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df